In [11]:
import pandas as pd
import numpy as np
import json
import matplotlib
import seaborn as sns
import nltk
import string
from nltk.corpus import stopwords
from sklearn.metrics import jaccard_score
from nltk.metrics import jaccard_distance
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize

# Performing EDA: Exploratory Data Analysis

In [12]:
#cur_dir = Path().resolve()
#print(cur_dir)
#path_to_file = Path.joinpath(cur_dir, 'potential_talents_OG.xlsx')
#join_path = cur_dir / 'potential_talents_OG.xlsx'
#print("the joined path =", join_path)

data = pd.read_excel("potential_talents_OG.xlsx")

In [13]:
# data = pd.read_excel(path_to_file)
# print(data)
print(data)
data.head()

      id                                          job_title  \
0      1  2019 C.T. Bauer College of Business Graduate (...   
1      2  Native English Teacher at EPIK (English Progra...   
2      3              Aspiring Human Resources Professional   
3      4             People Development Coordinator at Ryan   
4      5    Advisory Board Member at Celal Bayar University   
..   ...                                                ...   
99   100  Aspiring Human Resources Manager | Graduating ...   
100  101              Human Resources Generalist at Loparex   
101  102   Business Intelligence and Analytics at Travelers   
102  103                     Always set them up for Success   
103  104   Director Of Administration at Excellence Logging   

                                location connection  fit  
0                         Houston, Texas         85  NaN  
1                                 Kanada      500+   NaN  
2    Raleigh-Durham, North Carolina Area         44  NaN  
3      

,id,job_title,location,connection,fit
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN


In [14]:
data.describe() # the stats of the data, mean, mediun, max and min, and std value of the data

,id,fit
count,104.000000,0.0
mean,52.500000,NaN
std,30.166206,NaN
min,1.000000,NaN
25%,26.750000,NaN
50%,52.500000,NaN
75%,78.250000,NaN
max,104.000000,NaN


In [15]:
data.shape # checking the shape of the data

# balance data

(104, 5)

In [16]:
data['job_title']

0      2019 C.T. Bauer College of Business Graduate (...
1      Native English Teacher at EPIK (English Progra...
2                  Aspiring Human Resources Professional
3                 People Development Coordinator at Ryan
4        Advisory Board Member at Celal Bayar University
                             ...                        
99     Aspiring Human Resources Manager | Graduating ...
100                Human Resources Generalist at Loparex
101     Business Intelligence and Analytics at Travelers
102                       Always set them up for Success
103     Director Of Administration at Excellence Logging
Name: job_title, Length: 104, dtype: object

In [17]:
data['location']

0                           Houston, Texas
1                                   Kanada
2      Raleigh-Durham, North Carolina Area
3                            Denton, Texas
4                           İzmir, Türkiye
                      ...                 
99                Cape Girardeau, Missouri
100    Raleigh-Durham, North Carolina Area
101             Greater New York City Area
102               Greater Los Angeles Area
103                            Katy, Texas
Name: location, Length: 104, dtype: object

In [18]:
# seperate job_title and location by removing punctuation 

# List of column names to remove punctuation from
columns_to_clean = ['job_title']

# Function to remove punctuation
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator) if isinstance(text, str) else text

# Function to tokenize a string
def tokenize(text):
    tokenizer = RegexpTokenizer(r'\w+')
    return tokenizer.tokenize(text) if isinstance(text, str) else text

# Function to remove stop words from a list of tokens
def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    return [token for token in tokens if token.lower() not in stop_words]

# Function to lemmatize a list of tokens
def lemmatize(tokens):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(token) for token in tokens]

# Function to stem a list of tokens
def stem(tokens):
    stemmer = PorterStemmer()
    return [stemmer.stem(token) for token in tokens]

# Call the functions to the specified column
for column_name in columns_to_clean:
    data[column_name] = data[column_name].apply(remove_punctuation)
    data[column_name] = data[column_name].apply(tokenize)
    data[column_name] = data[column_name].apply(remove_stopwords)
    data[column_name] = data[column_name].apply(lemmatize)
    data[column_name] = data[column_name].apply(stem)

print(data[column_name])

json_data = data.to_json(orient='records')

# Specify the output file path
output_file = 'tokenized_file.json'

# Write the JSON data to a file
with open(output_file, 'w') as file:
    file.write(json_data)
# Save the modified DataFrame to a new Excel file
# data.to_excel('tokenized_file.xlsx', index=False) # save it as a json file

# read json file then convert it back into a df then take input (also apply all necessary steps for input)   
# then apply jaccard similarity 

0      [2019, ct, bauer, colleg, busi, graduat, magna...
1      [nativ, english, teacher, epik, english, progr...
2                    [aspir, human, resourc, profession]
3                        [peopl, develop, coordin, ryan]
4       [advisori, board, member, celal, bayar, univers]
                             ...                        
99     [aspir, human, resourc, manag, graduat, may, 2...
100                [human, resourc, generalist, loparex]
101                     [busi, intellig, analyt, travel]
102                                [alway, set, success]
103                    [director, administr, excel, log]
Name: job_title, Length: 104, dtype: object


To solve the respective error: AttributeError: 'list' object has no attribute 'translate'

In this updated code, we added checks to handle non-string values in the columns. The remove_punctuation and tokenize functions now check if the input text is a string before applying the corresponding operations. If the text is not a string (e.g., a list), it is returned as is without attempting to translate or tokenize it.

Tokenization: Breaks up the string into a list of words based on a specified pattern using Regular Expressions aka RegEx

Lemmatisation: mapping common words into one base.

Stemming: It cuts off prefixes and/or endings of words based on common ones.

In [19]:
# Read the JSON file
with open(output_file, 'r') as file:
    json_data = json.load(file)

In [21]:
df_json = pd.DataFrame(json_data)
read_data = pd.read_json('tokenized_file.json')
print(type[read_data['job_title']])

type[0      [2019, ct, bauer, colleg, busi, graduat, magna...
1      [nativ, english, teacher, epik, english, progr...
2                    [aspir, human, resourc, profession]
3                        [peopl, develop, coordin, ryan]
4       [advisori, board, member, celal, bayar, univers]
                             ...                        
99     [aspir, human, resourc, manag, graduat, may, 2...
100                [human, resourc, generalist, loparex]
101                     [busi, intellig, analyt, travel]
102                                [alway, set, success]
103                    [director, administr, excel, log]
Name: job_title, Length: 104, dtype: object]


### Applying same process as above but with inserting a free text

In [22]:
# Initialize the RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

# Initialize the WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# Initialize the PorterStemmer
stemmer = PorterStemmer()

# Insert a free text
free_text = input("Enter the free text: ")

# Tokenize the free text
tokens = tokenizer.tokenize(free_text)
# Remove punctuation
tokens_without_punctuation = [token for token in tokens if token.isalpha()]
# Remove stop words
stop_words = set(stopwords.words('english'))
tokens_without_stopwords = [token for token in tokens_without_punctuation if token.lower() not in stop_words]
# Lemmatize tokens
lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens_without_stopwords]
# Stem tokens
stemmed_tokens = [stemmer.stem(token) for token in lemmatized_tokens]

# Print the final tokens
print(stemmed_tokens)

Enter the free text: engineer
['engin']


# Performing Jaccard Similarity

In [23]:
# Extract the desired attribute values
attribute_values = read_data['job_title']
print(attribute_values)

0      [2019, ct, bauer, colleg, busi, graduat, magna...
1      [nativ, english, teacher, epik, english, progr...
2                    [aspir, human, resourc, profession]
3                        [peopl, develop, coordin, ryan]
4       [advisori, board, member, celal, bayar, univers]
                             ...                        
99     [aspir, human, resourc, manag, graduat, may, 2...
100                [human, resourc, generalist, loparex]
101                     [busi, intellig, analyt, travel]
102                                [alway, set, success]
103                    [director, administr, excel, log]
Name: job_title, Length: 104, dtype: object


In [27]:
# Calculate Jaccard similarity with each attribute value
jaccard_scores = []
for value in attribute_values:
    jaccard_sim = 1 - jaccard_distance(set(stemmed_tokens), set(value))
    print(jaccard_scores.append((value, jaccard_sim)))

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
